In [1]:
import json
import time
import os
import networkx as nx
from itertools import combinations
from datetime import datetime


#build permette di definire il punto di accesso da cui invocare i diversi endpoint delle YouTube API.
from googleapiclient.discovery import build

In [4]:
API_KEY = os.environ.get("API_KEY")

In [7]:
DEVELOPER_KEY = API_KEY
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

In [10]:
youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION, developerKey = DEVELOPER_KEY)

In [13]:
with open('videoToCommentersFinal.json', 'r') as f:
    videoToCommentersFinal = json.load(f)

In [16]:
videoToCommenters = videoToCommentersFinal

### Aggiungo attributi ai nodi: TITOLO, LIKES, VIEWS.

Salviamo la data di pubbliczione nel dict non come datetime ma come stringa (perché nel dict non si possono mettere i datetime). Nella fase di analisi la ritrasformeremo in datetime con datetime.strptime

In [19]:
def get_video_details(video_id):
    request = youtube.videos().list(
        part="snippet,statistics",
        id=video_id
    )
    response = request.execute()

    # Estrai i dettagli del video dalla risposta
    video_details = response['items'][0]
    title = video_details['snippet']['title']
    published_at = datetime.strptime(video_details['snippet']['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')
    likes = int(video_details['statistics'].get('likeCount', 0)) 
    views = int(video_details['statistics'].get('viewCount', 0))

    return {
        'title': title,
        'published_at': published_at,
        'likes': likes,
        'views': views
    }

scrivo in altro modo aggiungendo controllo eccezioni

In [ ]:
from datetime import datetime

def get_video_details(video_id):
    try:
        request = youtube.videos().list(
            part="snippet,statistics",
            id=video_id
        )
        response = request.execute()

        # Controlla se la risposta contiene elementi
        if 'items' in response and response['items']:
            # Estrai i dettagli del video dalla risposta
            video_details = response['items'][0]
            title = video_details['snippet']['title']
            published_at = datetime.strptime(video_details['snippet']['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')
            likes = int(video_details['statistics'].get('likeCount', 0)) 
            views = int(video_details['statistics'].get('viewCount', 0))
            comment_count = int(video_details['statistics'].get('commentCount', 0))  # Aggiungi il numero di commenti

            return {
                'title': title,
                'published_at': published_at,
                'likes': likes,
                'views': views,
                'comment_count': comment_count  # Aggiungi il numero di commenti
            }
        else:
            print(f"I dettagli per il video con ID {video_id} non sono stati trovati.")
            return None
    except Exception as e:
        print(f"Si è verificato un errore durante l'ottenimento dei dettagli per il video con ID {video_id}: {e}")
        return None



In [ ]:
# Ciclo attraverso ogni video nel dizionario e aggiungi i dettagli del video
for video_id, commenters in videoToCommenters.items():
    details = get_video_details(video_id)
    # Converti la data di pubblicazione in formato stringa
    details['published_at'] = str(details['published_at'])
    # Creo una nuova struttura dati contenente i dettagli del video e la lista di commentatori
    video_details = {
        'commenters': commenters,
        'details': details
    }
    # Aggiungi questa nuova struttura dati al dizionario videoToCommenters
    videoToCommenters[video_id] = video_details

# Salva il dizionario aggiornato come un file JSON
with open('videoToCommenters_with_details.json', 'w') as json_file:
    json.dump(videoToCommenters, json_file, indent=4)




In [ ]:
with open('videoToCommenters_with_details.json', 'r') as json_file:
    videoToCommenters_with_details = json.load(json_file)

In [ ]:
videoToCommenters_with_details

{'tem_iCzlHbI': {'commenters': [['@tintoriapodcast', 'Evvai'],
   ['@gabrielefregni',
    'Certo che i fan di tintoria sembrano un&#39; accozzaglia di imbecilli a giudicare dai commenti, non so se è perché hanno tredici anni o perché sono di Roma🤔'],
   ['@emanuelet1366',
    'Ma che cazzo dicono che coi tedeschi si parla solo di nazismo. Che provinciali ahah'],
   ['@julianhowald6268', 'Quando la puntata con fedez?'],
   ['@blue00magic99',
    'E niente, non bastano Tinti e Rapone a evitare il grattugiamento di scatole nel guardare questo podcast'],
   ['@pinomuscatelli6378', 'Tintoria &gt; Muschio selvaggio'],
   ['@skyall2261', 'Ahahahahahhaha la puntata più divertente di tutte!😂😂'],
   ['@DP-pf9ky',
    '<a href="https://www.youtube.com/watch?v=tem_iCzlHbI&amp;t=2436">40:36</a><br>fritz da cat <br>o<br>catz the fritz?'],
   ['@andrealarosa5682', 'Fate conto che Muschio Selvaggio sia il web...'],
   ['@angelorossi2471',
    'Io non ci sto capendo più niente, ma chi conduce muschio s

## Costruzione della rete

La procedura per la costruzione della rete è la seguente:

**Per ogni** coppia $(w,v)$ di video ottenibile dalla collezione dei video:<br>
&ensp;&thinsp;Calcolare il numero di commentantori in comune tra $w$ e $v$<br>
&ensp;&thinsp;Inserire nella rete un arco tra $w$ e $v$ il cui peso è il numero di co-commentatori

Dato un insieme/sequenza di elementi in Python possiamo agilmente farci restituire tutte le possibili coppie che posso comporre con gli elementi della sequenza utilizzando la funzione `combinations` del modulo `itertools`.

Importiamo quindi la funzione, la cui documentazione è disponibile [qui](https://docs.python.org/3/library/itertools.html#itertools.combinations).

Generiamo quindi le possibili coppie di video ed implementiamo la precedente procedura. Prima di implementare la procedura si costruisce un grafo vuoto, in questo caso non orientato.

In [ ]:
# grafo vuoto non orientato
muschio_net = nx.Graph()

In [ ]:
# Aggiungi nodi al grafo con gli attributi
for video_id, data in videoToCommenters_with_details.items():
    # Aggiungi i dettagli del video come attributi del nodo
    muschio_net.add_node(video_id, **data['details'])

# Aggiungi archi pesati per la condivisione di commentatori
for video1, video2 in combinations(videoToCommenters_with_details.keys(), 2):
    # Ottieni gli ID dei commentatori per i due video
    commenters1 = set(commenter for commenter, _ in videoToCommenters_with_details[video1]['commenters'])
    commenters2 = set(commenter for commenter, _ in videoToCommenters_with_details[video2]['commenters'])
    # Calcola gli ID dei commentatori in comune
    common_commenters = commenters1.intersection(commenters2)
    # Aggiungi un arco solo se ci sono commentatori in comune
    if common_commenters:
        muschio_net.add_edge(video1, video2)

In [ ]:
# Salva il grafo in formato GEXF con gli attributi
nx.write_gexf(muschio_net, 'muschio_net.gexf')

prima cosa da fare: generare tutte le combinazioni

In [ ]:
muschio_net.size(), muschio_net.order()

(3112, 100)

CARICO IL MIO GRAFO

In [ ]:
muschio_net = nx.read_gexf('muschio_net.gexf')

prendo il file che ho generato e lo apro con Gephi per visualizzare il grafo